In [1]:
import tensorflow as tf
import pickle
import os
import shutil
import tempfile
import datetime
from glob import iglob
import time
# import attention
from collections import deque
import pickle
import pyreader
import numpy as np
import csv
from batchmake import Batcher
import pprint
tf.set_random_seed(777)

In [2]:
datas= 'data_samples/'
vocabulary = 'vocab.csv'
vocabs = []
with open(vocabulary, 'r', newline='', encoding='utf-8') as vocab:
    words = csv.reader(vocab)
    for word in words:
        vocabs.append(word)
print(len(vocabs))

22669


In [3]:
hidden_size=32
sequence=5
embedding_dim=50
attention_size = 50
batch_size=16
vocab_size=len(vocabs)
grucell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size)
X = tf.placeholder(tf.int32, [None, sequence], name='inputs_xdata')
# Y = tf.placeholder(tf.float32, [None, sequence], name='targets_ydata')
Y = tf.placeholder(tf.int32, [None, sequence], name='targets_ydata')
seq_len = tf.placeholder(tf.int32, [None], name='seq_len')

embedding_variable = tf.Variable(tf.random_uniform([vocab_size, embedding_dim],-1.0,1.0), trainable=True)
# inputs_ = tf.transpose(X)
batch_embedded = tf.nn.embedding_lookup(embedding_variable, X)

print(batch_embedded)
with tf.variable_scope('RNN'):
    outputs, states = tf.nn.dynamic_rnn(grucell,
                                        inputs=batch_embedded,
                                        sequence_length=seq_len,
                                        dtype=tf.float32)

Tensor("embedding_lookup:0", shape=(?, 5, 50), dtype=float32)


In [4]:
outputs,states

(<tf.Tensor 'RNN/rnn/transpose:0' shape=(?, 5, 32) dtype=float32>,
 LSTMStateTuple(c=<tf.Tensor 'RNN/rnn/while/Exit_2:0' shape=(?, 32) dtype=float32>, h=<tf.Tensor 'RNN/rnn/while/Exit_3:0' shape=(?, 32) dtype=float32>))

In [ ]:

tf.einsum('')

In [6]:
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, vocab_size, activation_fn=None)

In [10]:
outputs = tf.reshape(outputs, [batch_size, sequence, vocab_size])
outputs

<tf.Tensor 'Reshape_3:0' shape=(16, 5, 22669) dtype=float32>

In [101]:
Y

<tf.Tensor 'targets_ydata:0' shape=(?, 5) dtype=int32>

In [20]:
weights = tf.ones([batch_size, sequence])

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)


In [21]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(mean_loss)

In [22]:
file_name_queue = tf.train.string_input_producer(['inputs'])
reader = tf.TFRecordReader()
_, single_x = reader.read(file_name_queue)

context_features = {
    "length": tf.FixedLenFeature([], dtype=tf.int64)
}
sequence_features = {
    "tokens": tf.FixedLenSequenceFeature([], dtype=tf.int64),
    "labels": tf.FixedLenSequenceFeature([], dtype=tf.int64)
}


context_parsed, sequence_parsed = tf.parse_single_sequence_example(serialized=single_x,
                                 context_features=context_features,
                                 sequence_features=sequence_features
                                )
batch_seq, batch_x, batch_y = tf.train.batch(tensors=[context_parsed['length'],sequence_parsed['tokens'],sequence_parsed['labels']],
                                                     batch_size=batch_size, 
                                                     capacity=5000,
                                                     num_threads=1,
                                             dynamic_pad=True
                                                    )

In [25]:
num_epochs = 10
delta = 0.5

Ses = tf.Session()
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=Ses, coord=coord)
Ses.run(tf.global_variables_initializer())

print('start learning!')

for epoch in range(num_epochs):
    
    print('{} start'.format(epoch))
    
    loss_train = 0
    loss_test = 0
    accuracy_train = 0
    accuracy_test =0
    
    
#     for i in range(1000):
    i=0
#     try:
    while True:
        seq_, x_in, y_in = Ses.run([batch_seq, batch_x, batch_y])
        feed_data = {X:x_in, Y:y_in, seq_len:seq_}
#         print(feed_data)
#         lo, acc, opt = Ses.run([loss, accuracy, optimizer], feed_dict=feed_data)
        lo, opt = Ses.run([loss, optimizer], feed_dict=feed_data)
#         accuracy_train+=acc
        loss_train = lo * delta + loss_train * (1-delta)
        i+=1
        if i%100 ==0:
            print(' {} : loss {} acc {}'.format(i, lo, accuracy_train))
#     except:
#         print('batch : ',i)
#         accuracy_train /= 
        

start learning!
0 start
 100 : loss 6.166654586791992 acc 0
 200 : loss 6.080562114715576 acc 0
 300 : loss 4.802710056304932 acc 0
 400 : loss 5.169119834899902 acc 0
 500 : loss 5.313636302947998 acc 0
 600 : loss 4.895544052124023 acc 0
 700 : loss 5.743092060089111 acc 0
 800 : loss 6.028770446777344 acc 0
 900 : loss 4.821763038635254 acc 0
 1000 : loss 5.6741790771484375 acc 0
 1100 : loss 5.161557197570801 acc 0
 1200 : loss 5.163616180419922 acc 0
 1300 : loss 3.7881648540496826 acc 0
 1400 : loss 6.136954307556152 acc 0
 1500 : loss 4.925378799438477 acc 0
 1600 : loss 4.46866512298584 acc 0
 1700 : loss 4.525747776031494 acc 0
 1800 : loss 5.006093502044678 acc 0
 1900 : loss 3.9451472759246826 acc 0
 2000 : loss 4.064728736877441 acc 0
 2100 : loss 4.5308518409729 acc 0
 2200 : loss 5.052001476287842 acc 0
 2300 : loss 4.174595832824707 acc 0
 2400 : loss 6.403733730316162 acc 0
 2500 : loss 4.8307013511657715 acc 0
 2600 : loss 2.919546604156494 acc 0
 2700 : loss 4.2431755

KeyboardInterrupt: 

In [38]:
# feed_data = {X:}
opt = Ses.run([outputs], feed_dict=feed_data)
print(opt[0])


[[[-13.29698753   1.50603712   4.18791246 ..., -13.349123   -13.3751421
   -13.1564827 ]
  [-17.8072567    1.72352362   6.62874079 ..., -17.88330269 -17.91202545
   -17.60957336]
  [ -8.10572243   0.96238852   3.64221597 ...,  -8.1366148   -8.14382172
    -8.0562582 ]
  [ -9.59585762   1.19165468   3.02162147 ...,  -9.67808628  -9.68119144
    -9.53866673]
  [-16.43118095   0.92567235   6.14692831 ..., -16.50385666 -16.52963066
   -16.25689888]]

 [[-12.68002701   2.50740552   4.95650816 ..., -12.72162056 -12.75607491
   -12.54108429]
  [-10.22591019   0.89076328   4.29207182 ..., -10.25300407 -10.27827072
   -10.15265274]
  [ -7.87790489   1.41231561   1.4872905  ...,  -7.92313194  -7.90665865
    -7.81639719]
  [-14.17497826   1.20529819   3.52974939 ..., -14.25171852 -14.27195454
   -14.0370369 ]
  [ -8.08020782   3.09560299   1.16355121 ...,  -8.10146046  -8.13920975
    -8.01443386]]

 [[-11.9107151    2.65493131   2.21742368 ..., -11.96892166 -11.97667599
   -11.79452229]
  [ -8.

ValueError: Tried to convert 'input' to a tensor and failed. Error: Argument must be a dense tensor: [array([[[-13.29698753,   1.50603712,   4.18791246, ..., -13.349123  ,
         -13.3751421 , -13.1564827 ],
        [-17.8072567 ,   1.72352362,   6.62874079, ..., -17.88330269,
         -17.91202545, -17.60957336],
        [ -8.10572243,   0.96238852,   3.64221597, ...,  -8.1366148 ,
          -8.14382172,  -8.0562582 ],
        [ -9.59585762,   1.19165468,   3.02162147, ...,  -9.67808628,
          -9.68119144,  -9.53866673],
        [-16.43118095,   0.92567235,   6.14692831, ..., -16.50385666,
         -16.52963066, -16.25689888]],

       [[-12.68002701,   2.50740552,   4.95650816, ..., -12.72162056,
         -12.75607491, -12.54108429],
        [-10.22591019,   0.89076328,   4.29207182, ..., -10.25300407,
         -10.27827072, -10.15265274],
        [ -7.87790489,   1.41231561,   1.4872905 , ...,  -7.92313194,
          -7.90665865,  -7.81639719],
        [-14.17497826,   1.20529

In [78]:
predict = tf.nn.top_k(outputs,5)

In [98]:
seq_, x_in, y_in = Ses.run([batch_seq, batch_x, batch_y])
feed_data = {X:x_in, seq_len:seq_, Y:y_in}
pr = Ses.run(predict, feed_dict=feed_data)
feed_data

{<tf.Tensor 'inputs_xdata:0' shape=(?, 5) dtype=int32>: array([[6282,    4,    8,    2,    9],
        [5482,    7,   15,    5,   37],
        [   5,  186,    3,    3,    1],
        [  23,    3, 1216,    6,  943],
        [   4,    4,    6,   31,    4],
        [   2,   16,    3, 5482,   25],
        [  31,    4,    8,    2,    9],
        [  15,    5,   37,    5,   41],
        [   3,    3,    1,   23,    3],
        [ 482,    6, 1216,    6,  413],
        [   4,    4,    6,   15,    5],
        [  72,    4,    2,   18,    2],
        [  10,   10,   15,    5,   80],
        [   5,   84,    5,  144,    3],
        [   4,    2,  294,    7,    3],
        [   1,   23,    3, 1216,    6]], dtype=int64),
 <tf.Tensor 'seq_len:0' shape=(?,) dtype=int32>: array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5], dtype=int64),
 <tf.Tensor 'targets_ydata:0' shape=(?, 5) dtype=int32>: array([[   4,    8,    2,    9, 5482],
        [   7,   15,    5,   37,    5],
        [ 186,    3,    3,    1,   

In [83]:
prob = pr[0]
probidx = pr[1]
idx_to_word = []
with open(vocabulary, 'r', newline='', encoding='utf-8') as vocab:
    words = csv.reader(vocab)
    for word in words:
        idx_to_word.append(word)

In [100]:
probidx
print(feed_data[X], feed_data[Y])
for line in probidx:
    for idx in line:
#         for each_idx in idx:
        print(idx_to_word[idx[0]], idx[0])
    print()
# predic_word = [idx_to_word[i] for i in probidx[0]]

[[6282    4    8    2    9]
 [5482    7   15    5   37]
 [   5  186    3    3    1]
 [  23    3 1216    6  943]
 [   4    4    6   31    4]
 [   2   16    3 5482   25]
 [  31    4    8    2    9]
 [  15    5   37    5   41]
 [   3    3    1   23    3]
 [ 482    6 1216    6  413]
 [   4    4    6   15    5]
 [  72    4    2   18    2]
 [  10   10   15    5   80]
 [   5   84    5  144    3]
 [   4    2  294    7    3]
 [   1   23    3 1216    6]] [[   4    8    2    9 5482]
 [   7   15    5   37    5]
 [ 186    3    3    1   23]
 [   3 1216    6  943    4]
 [   4    6   31    4    2]
 [  16    3 5482   25   31]
 [   4    8    2    9   15]
 [   5   37    5   41    3]
 [   3    1   23    3  482]
 [   6 1216    6  413    4]
 [   4    6   15    5   72]
 [   4    2   18    2   10]
 [  10   15    5   80    5]
 [  84    5  144    3    4]
 [   2  294    7    3    1]
 [  23    3 1216    6  482]]
['from'] 9
['if'] 3
['self'] 1
['0'] 5
['if'] 3

['if'] 3
['if'] 3
['0'] 5
['if'] 3
['if'] 3

['self']